### Linear optimization

First we'll list out all the probabilities we need: p(x,y|z) for each x,y,z combination.

In [17]:
p_states

,"P( X, Y | Z )"
state,
control/untreated/bad,0.315285
control/untreated/good,0.043756
control/treated/bad,0.323676
control/treated/good,0.317283
treatment/untreated/bad,0.019820
treatment/untreated/good,0.670671
treatment/treated/bad,0.167968
treatment/treated/good,0.141542


Now we can frame our linear programming problem.

We're maximizing our expression of an ATE given the constraints at hand.

In [18]:
import pulp

# first we set up 'problems', objects that
# can take an objective like 'maximize' as well as constraints

max_problem = pulp.LpProblem(
    "max ATE", pulp.LpMaximize
)

We can now specify variables, symbols that the linear program will know to vary in order to maximize or minimize our objective function. These variables are the distribution of **U**: the probabilities of belonging to one of the archetypal (Compliance, Response) partitions.

In [19]:
# Linear optimization ranges over possible variables
# to find those hidden variables that maximize
# or minimize our objective.

# Our variables 'q' are the the probability of
# being in one of the partitions

partition_names = [
    "/".join([compliance, response])
    for compliance, response in partition_types
]

# Notice that we put a lower bound of 0 on each
# p(partition) value.
# This is because probability values cannot be negative.
q = {
    partition: pulp.LpVariable(partition, lowBound=0)
    for partition in partition_names
}

We can add constraints on these variables as well, by simply adding them to the problem we've framed!

An obvious constraint: the probabilities of being in each partition must sum up to 1:

In [20]:
# Since our vars are probabilities
# the sum of them should all be under 1.

# This '+=' operation is adding this sum constraint
# to the linear programming problem.

max_problem += sum([v for k, v in q.items()]) == 1

The real key constraint to add: the probabilities of being in a paritition are linked directly to the experimental data we observe:

In [21]:
# Let's set the relationship between the distribution of U
# and the probabilities we observe. The variable name
# scheme is p_Z_X_Y


p_treatment_untreated_bad = (
    q["never_taker/never_better"]
    + q["defier/never_better"]
    + q["never_taker/helped"]
    + q["defier/helped"]
)

p_treatment_untreated_good = (
    q["never_taker/always_better"]
    + q["defier/always_better"]
    + q["never_taker/hurt"]
    + q["defier/hurt"]
)

p_treatment_treated_bad = (
    q["always_taker/never_better"]
    + q["complier/never_better"]
    + q["always_taker/hurt"]
    + q["complier/hurt"]
)

p_treatment_treated_good = (
    q["always_taker/always_better"]
    + q["complier/always_better"]
    + q["always_taker/helped"]
    + q["complier/helped"]
)

p_control_untreated_bad = (
    q["never_taker/never_better"]
    + q["complier/never_better"]
    + q["never_taker/helped"]
    + q["complier/helped"]
)

p_control_untreated_good = (
    q["never_taker/always_better"]
    + q["complier/never_better"]
    + q["never_taker/hurt"]
    + q["complier/hurt"]
)

p_control_treated_bad = (
    q["always_taker/never_better"]
    + q["defier/never_better"]
    + q["always_taker/hurt"]
    + q["defier/hurt"]
)

p_control_treated_good = (
    q["always_taker/always_better"]
    + q["defier/always_better"]
    + q["always_taker/helped"]
    + q["defier/helped"]
)

In [22]:
# I now apply these linear relationships between U
# and P(X,Y|Z) as constraints on the LP problem.

# These '+=' operations are new constraints I am 
# adding one-by-one to the 'max_problem' linear programming 
# problem.

max_problem += (
    p_treatment_untreated_bad
    == p_states.loc["treatment/untreated/bad"]
)
max_problem += (
    p_treatment_treated_good
    == p_states.loc["treatment/treated/good"]
)
max_problem += (
    p_treatment_treated_bad
    == p_states.loc["treatment/treated/bad"]
)

max_problem += (
    p_control_untreated_bad
    == p_states.loc["control/untreated/bad"]
)
max_problem += (
    p_control_treated_good
    == p_states.loc["control/treated/good"]
)
max_problem += (
    p_control_treated_bad
    == p_states.loc["control/treated/bad"]
)


# I leave some constraints out because it *over*constrains
# the problem and makes it impossible to solve
# It turns out that the other constraints actually imply
# these two since they are complimentary probabilities,
# so we can just leave them commented out

# max_problem += p_control_untreated_good == p_states.loc['control/untreated/good']
# max_problem += p_treatment_untreated_good == p_states.loc['treatment/untreated/good']

With constraints set, all that remains to do set the objective function, our ATE.

$$ATE = P(helped) - P(hurt)$$

where $P(helped)$ is the probability of being in a partition with a 'helped' response type and $P(hurt)$ is the probability of being in a partition with a 'hurt' response type.

Then we can just hit `solve()`.

In [23]:
max_problem += (
    q["complier/helped"]
    + q["defier/helped"]
    + q["always_taker/helped"]
    + q["never_taker/helped"]
    - q["complier/hurt"]
    - q["defier/hurt"]
    - q["always_taker/hurt"]
    - q["never_taker/hurt"]
)

In [24]:
pulp.LpStatus[max_problem.solve()]

'Optimal'

The problem's been optimized, and now we can see what U looks like in the best case scenario.

In [25]:
q_max = {
    partition: pulp.value(partition_p)
    for partition, partition_p in q.items()
}

# display

pd.DataFrame(q_max, index=["p(U=u)"]).T

,p(U=u)
always_taker/always_better,0.000000
always_taker/helped,0.000000
always_taker/hurt,0.000000
always_taker/never_better,0.014045
complier/always_better,0.000000
complier/helped,0.141542
complier/hurt,0.000000
complier/never_better,0.153923
defier/always_better,0.317283
defier/helped,0.000000


Given these we can calculate our best-case ATE.

In [26]:
ate = (
    lambda q: q["complier/helped"]
    + q["defier/helped"]
    + q["always_taker/helped"]
    + q["never_taker/helped"]
    - q["complier/hurt"]
    - q["defier/hurt"]
    - q["always_taker/hurt"]
    - q["never_taker/hurt"]
)

best_case_ate = ate(q_max)

# display
print("Best-Case ATE: %6.4f" % best_case_ate)

Best-Case ATE: -0.1483
